### "СompressFun! или методы сжатия для самых маленьких"

**RLE (Run-Length Encoding) кодирование длин серий**

**Задача:**
Укоротить последовательность (количество повторений символа + повторяющийся символ)   
ААААА -> 5A   
АААААBBCCCCHHHHH -> 5A2B4C5H

In [30]:
# Пример: АААААGGCCCCNHHHHH
def RLE(data):
    encodedList = []
    lastSymbol = data.pop(0)    # Берём и удаляем первый символ входных данных для дальнейшего сравнения
    k = 1
    output = True
    lenData = len(data)
    for i in range(lenData):
        output = True
        if (lastSymbol == data[i]):
            k += 1
            output = False
        if (output == True):
            if (k > 2):
                encodedList.append(str(k) + str(lastSymbol))
            if (k == 2):
                encodedList.append(str(lastSymbol + lastSymbol))
            if (k == 1):
                encodedList.append(str(lastSymbol))
            lastSymbol = data[i]
            k = 1
        if (i == (lenData - 1)):
            encodedList.append(str(lastSymbol))
    final = ''.join(encodedList)
    return final

#print(RLE(list(input())))
print("АААААGGCCCCNHHHHH")
print(RLE(list("АААААGGCCCCNHHHHH")))

АААААGGCCCCNHHHHH
5АGG4CNH


Неплохой способ "сжатия" с длинными последовательными повторениями символов, но этот способ малоэффективен в случаях символов N и G из примера. "GG" меняется на "2G", что бессмысленно, а "N" меняется на "1N", что вообще никуда не годиться. **НО**, ... про доп. байт

**BWT (Burrows-Wheeler transform)**    
Мы сделаем лучше. **BWT** позволяет с помощью точного алгоритма сгруппировать одинаковые части последовательности для дальнейшего использования **RLE**.

**Без BWT:**    
ABACABA -> 1A1B1A1C1A1B1A

**С BWT:**    
ABACABA -> CBBAAAA
CBBAAAA -> 1C2B4A

На более больших последовательностях выгода будет существенно больше.

**Описание алгоритма**  
Преобразование выполняется в три этапа:

1. Составляется таблица всех циклических сдвигов входной строки.
2. Производится лексикографическая (в алфавитном порядке) сортировка строк таблицы.
3. В качестве выходной строки выбирается последний столбец таблицы преобразования и номер строки, совпадающей с исходной.    

**Описание взято с Wiki университета ИТМО**

In [31]:
def printList(inList):
    for part in inList:
        print(part)
    print()

# Пример: АНАНАС    
def BWT(data):
    if (debug):
        print()
    lenData = len(data)
    data.append("■") #alt+254 (■ = 254)
    sortList = []
    for cycle in data:
        sortList.append(''.join(data))
        data.append(data.pop(0))  
    if (debug): 
        printList(sortList)
    sortList.sort()
    if (debug):
        printList(sortList)
    finalList = []
    for cycle in sortList:
        finalList.append(cycle[lenData]) 
    final = ''.join(finalList)
    return final

debug = 1   
#print(BWT(list(input())))
print("АНАНАС")
print(BWT(list("АНАНАС")))

АНАНАС

АНАНАС■
НАНАС■А
АНАС■АН
НАС■АНА
АС■АНАН
С■АНАНА
■АНАНАС

АНАНАС■
АНАС■АН
АС■АНАН
НАНАС■А
НАС■АНА
С■АНАНА
■АНАНАС

■ННАААС


In [32]:
# Пример: ■ННАААС  
def BWTDecode(data):
    n = len(data)
    sortList = data.copy() 
    sortList.sort()
    for x in range(n - 1):
        for y in range(n):
            last = sortList[y]
            sortList.pop(y)
            sortList.insert(y, str(data[y]) + str(last)) 
        sortList.sort()
        if (debug):
            print(sortList)
    for variant in sortList:
        if (variant[0] == "■"):
            final = variant
    final = final[1:]
    return final
    
debug = 1
#print(BWTDecode(list(input())))
print("■ННАААС")
print(BWTDecode(list("■ННАААС")))

■ННАААС
['АН', 'АН', 'АС', 'НА', 'НА', 'С■', '■А']
['АНА', 'АНА', 'АС■', 'НАН', 'НАС', 'С■А', '■АН']
['АНАН', 'АНАС', 'АС■А', 'НАНА', 'НАС■', 'С■АН', '■АНА']
['АНАНА', 'АНАС■', 'АС■АН', 'НАНАС', 'НАС■А', 'С■АНА', '■АНАН']
['АНАНАС', 'АНАС■А', 'АС■АНА', 'НАНАС■', 'НАС■АН', 'С■АНАН', '■АНАНА']
['АНАНАС■', 'АНАС■АН', 'АС■АНАН', 'НАНАС■А', 'НАС■АНА', 'С■АНАНА', '■АНАНАС']
АНАНАС


In [33]:
def Division(allSymbols, n):   
    part = []
    for i in range(0, len(allSymbols), n):
        part.append(allSymbols[i:i+n])
    return part

In [35]:
# Пример: АНАНАСАJАJАСАGАGАС
debug = 0
listParts = Division(input(), 30)
print(listParts)
for part in listParts:
    appliedBWT = BWT(list(part))
    #print(appliedBWT)
    appliedRLE = RLE(list(appliedBWT))
    print(appliedRLE, end='')

АНАНАСАJАJАСАGАGАС
['АНАНАСАJАJАСАGАGАС']
4АСGСJ■НJНG5АС

In [54]:
# Полезная фича 1
str = "Hello"
print(str[0:-2])
print(str[:-2]) # Так тоже можно, но читаемость хуже
print(str[2:-2])
print(str[2:5])
print(str[1:])

Hel
Hel
l
llo
ello


In [101]:
# Полезная фича 2
myList = [4, 6, 3, 8]
print(myList)
myList.pop(2)
myList.insert(2, 9)
print(myList)

[4, 6, 3, 8]
[4, 6, 9, 8]
